# GitHub Pull Requests dataset
## The dataset
The dataset used for the base level knowledge augmentation can be found at
https://www.kaggle.com/datasets/pelmers/github-public-pull-request-comments.

It contains JSONs of Pull Request (file paths, comments, and diffs, among other things) from  mined from permissively-licensed GitHub public projects with at least 25 stars and 25 pull requests submitted at the time of access and covers Go, Java, JavaScript, TypeScript, and Python.

## Data pipeline
The task of this notebook is to ingest each Pull Request in the dataset, embed it using a feature extraction model, and upload it to a vector database in order to enable Retrieval Augmented Generation (RAG).
Given the size of the overall dataset (over 30GB), this notebook will focus on the JavaScript portion of the dataset as a proof of concept.

### Data treatment
Some level of treatment of the data is necessary given that:
- All the data is present in a single json file, which is a barrier to parallelization of the embeddings upload
- And although they fit the 25 start and 25 pull request requirement to be mined, for various reasons including PRs not being related to code covered by the dataset, some repositories present in the dataset had no data

So in this notebook the subdataset is split into a json file for each repository in a manner that will facilitate future data processing pipelines of those files.

In [1]:
from tqdm.auto import tqdm
import pandas as pd

df = pd.read_json('dataset/mined-comments-25stars-25prs-JavaScript.json/mined-comments-25stars-25prs-JavaScript.json', orient='index')

In [2]:
# checking how many rows it had before
df.shape

(28835, 9224)

In [3]:
# remove any row that has not data
df = df.dropna(axis=0, how='all')

In [4]:
# checking how many rows it had after
df.shape

(7366, 9224)

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,9214,9215,9216,9217,9218,9219,9220,9221,9222,9223
trekhleb/javascript-algorithms,{'html_url': 'https://github.com/trekhleb/java...,{'html_url': 'https://github.com/trekhleb/java...,{'html_url': 'https://github.com/trekhleb/java...,{'html_url': 'https://github.com/trekhleb/java...,{'html_url': 'https://github.com/trekhleb/java...,{'html_url': 'https://github.com/trekhleb/java...,{'html_url': 'https://github.com/trekhleb/java...,{'html_url': 'https://github.com/trekhleb/java...,{'html_url': 'https://github.com/trekhleb/java...,{'html_url': 'https://github.com/trekhleb/java...,...,None,None,None,None,None,None,None,None,None,None
airbnb/javascript,{'html_url': 'https://github.com/airbnb/javasc...,{'html_url': 'https://github.com/airbnb/javasc...,{'html_url': 'https://github.com/airbnb/javasc...,{'html_url': 'https://github.com/airbnb/javasc...,{'html_url': 'https://github.com/airbnb/javasc...,{'html_url': 'https://github.com/airbnb/javasc...,{'html_url': 'https://github.com/airbnb/javasc...,{'html_url': 'https://github.com/airbnb/javasc...,{'html_url': 'https://github.com/airbnb/javasc...,{'html_url': 'https://github.com/airbnb/javasc...,...,None,None,None,None,None,None,None,None,None,None
twbs/bootstrap,{'html_url': 'https://github.com/twbs/bootstra...,{'html_url': 'https://github.com/twbs/bootstra...,{'html_url': 'https://github.com/twbs/bootstra...,{'html_url': 'https://github.com/twbs/bootstra...,{'html_url': 'https://github.com/twbs/bootstra...,{'html_url': 'https://github.com/twbs/bootstra...,{'html_url': 'https://github.com/twbs/bootstra...,{'html_url': 'https://github.com/twbs/bootstra...,{'html_url': 'https://github.com/twbs/bootstra...,{'html_url': 'https://github.com/twbs/bootstra...,...,None,None,None,None,None,None,None,None,None,None
30-seconds/30-seconds-of-code,{'html_url': 'https://github.com/30-seconds/30...,{'html_url': 'https://github.com/30-seconds/30...,{'html_url': 'https://github.com/30-seconds/30...,{'html_url': 'https://github.com/30-seconds/30...,{'html_url': 'https://github.com/30-seconds/30...,{'html_url': 'https://github.com/30-seconds/30...,{'html_url': 'https://github.com/30-seconds/30...,{'html_url': 'https://github.com/30-seconds/30...,{'html_url': 'https://github.com/30-seconds/30...,{'html_url': 'https://github.com/30-seconds/30...,...,None,None,None,None,None,None,None,None,None,None
d3/d3,{'html_url': 'https://github.com/d3/d3/pull/10...,{'html_url': 'https://github.com/d3/d3/pull/10...,{'html_url': 'https://github.com/d3/d3/pull/10...,{'html_url': 'https://github.com/d3/d3/pull/10...,{'html_url': 'https://github.com/d3/d3/pull/10...,{'html_url': 'https://github.com/d3/d3/pull/10...,{'html_url': 'https://github.com/d3/d3/pull/10...,{'html_url': 'https://github.com/d3/d3/pull/10...,{'html_url': 'https://github.com/d3/d3/pull/10...,{'html_url': 'https://github.com/d3/d3/pull/10...,...,None,None,None,None,None,None,None,None,None,None


In [6]:
df.index

Index(['trekhleb/javascript-algorithms', 'airbnb/javascript', 'twbs/bootstrap',
       '30-seconds/30-seconds-of-code', 'd3/d3', 'facebook/react',
       'facebook/react-native', 'facebook/create-react-app', 'axios/axios',
       'vercel/next.js',
       ...
       'ezsystems/ezoe', 'mac-/hapi-statsd', 'twinlabs/forum',
       'strongloop/loopback-sdk-angular-cli', 'bem/bem-mvc',
       'manguezal/manguezal.github.com', 'Ember-SC/peepcode-ordr-test',
       'apache/cordova-coho', 'papandreou/node-jpegtran',
       'meetup/meetup-web-components'],
      dtype='object', length=7366)

In [7]:
df.columns

RangeIndex(start=0, stop=9224, step=1)

In [8]:
# saving each row of the df to a dataframe
dfs = []
for i in tqdm(range(len(df)), 'Separating data by repository'):
    dfs.append(pd.DataFrame(df.iloc[i]).T)

Separating data by repository:   0%|          | 0/7366 [00:00<?, ?it/s]

In [9]:
# deleting the df variable to save memory
del df

In [10]:
# dropping cells with empty data in each dataframe
for i in tqdm(range(len(dfs)), 'Removing empty cells'):
    dfs[i] = dfs[i].dropna(axis='columns', how='all')

Removing empty cells:   0%|          | 0/7366 [00:00<?, ?it/s]

In [11]:
# ordering the dataframes by the number of PRs of each repository in descending order
dfs = sorted(dfs, key=lambda x: len(x.columns), reverse=True)

In [12]:
dfs[len(dfs) - 1]

,0
manguezal/manguezal.github.com,{'html_url': 'https://github.com/manguezal/man...


In [13]:
dfs[0][0][0]

{'html_url': 'https://github.com/plotly/plotly.js/pull/1#discussion_r44735997',
 'path': 'devtools/test_dashboard/server.js',
 'line': 36,
 'body': 'the test dashboard server script uses the same watch-bundling machinery as `npm run watch` \n:palm_tree: :palm_tree: \n',
 'user': 'etpinard',
 'diff_hunk': "@@ -1,89 +1,53 @@\n-var http = require('http');\n-var ecstatic = require('ecstatic');\n-var browserify = require('browserify');\n-var open = require('open');\n var fs = require('fs');\n-var watchify = require('watchify');\n+var http = require('http');\n var path = require('path');\n-var outpipe = require('outpipe');\n-var outfile = path.join(__dirname, '../shelly/plotlyjs/static/plotlyjs/build/plotlyjs-bundle.js');\n-\n-var testFile = './test';\n-\n-switch(process.argv[2]) {\n-  case 'geo':\n-    testFile = './test-geo';\n-  break;\n-  case '2d':\n-    testFile = './test-2d';\n-  break;\n-}\n-\n-console.log('using ' + testFile);\n-\n-var b = browserify(path.join(__dirname, '../shelly/

In [14]:
for i in tqdm(range(len(dfs)), 'Replace every / in the index with a -'):
    dfs[i].index = dfs[i].index.str.replace('/', '@')

Replace every / in the index with a -:   0%|          | 0/7366 [00:00<?, ?it/s]

In [15]:
# create repo-split folder
import os

if not os.path.exists('dataset/mined-comments-25stars-25prs-JavaScript.json/repo-split'):
    os.makedirs('dataset/mined-comments-25stars-25prs-JavaScript.json/repo-split')

# saving each df to a json file with the repository name as the file name with i in the name so we know the order from the most to the least columns
for i in tqdm(range(len(dfs)), 'Saving dataframes to json files'):
    dfs[i].to_json('dataset/mined-comments-25stars-25prs-JavaScript.json/repo-split/' + str(i+1) + '-' + dfs[i].index[0] + '.json')

Saving dataframes to json files:   0%|          | 0/7366 [00:00<?, ?it/s]

In [16]:
# deleting the dfs variable to save memory
del dfs